In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor
import torch
import torch_geometric
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pathlib import Path

In [26]:
class EEGGraphConvNetLSTM(nn.Module):
    
    def __init__(self, reduced_sensors=True, sfreq=None, batch_size=256, **kwargs):
        super(EEGGraphConvNetLSTM, self).__init__()
        # Define and initialize hyperparameters
        self.sfreq = sfreq
        self.batch_size = batch_size
        #self.input_size = 8 if reduced_sensors else 62
        
        # Layers definition
        input_size = kwargs.pop("input_size", 1280)
        hidden_dim = 320
        
        # Graph convolutional layers
        self.conv1 = GCNConv(input_size, 640, cached=True, normalize=False)
        self.conv2 = GCNConv(640, 512, cached=True, normalize=False)
        self.conv3 = GCNConv(512, 256, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm4 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        self.lstm = nn.LSTM(256, 256, 1, dropout=0.2)
        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm3(self.conv3(x, edge_index, edge_weigth)), negative_slope=0.01)
        x, _ = self.lstm(x)
        #x = F.leaky_relu(self.batch_norm4(self.conv4(x, edge_index, edge_weigth)), negative_slope=0.01)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        add_pool = global_add_pool(x, batch=batch)
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(add_pool), negative_slope=0.01)
        out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc3(out))
        return out
        

In [27]:
def create_corrected_data_list(path):
    data_list = list()
    for file in path.iterdir():
        data_list.append(torch.load(file))
        
    corrected_data_list = list()
    
    for data in data_list:
    # print(data)
        data = torch_geometric.data.Data(
            x=torch.tensor(data.x),
            edge_index=torch.tensor(data.edge_index),
            edge_attr=torch.tensor(data.edge_attr),
            label=torch.tensor(data.label),
        )
        corrected_data_list.append(data)
        
    rm = [
      7,
      14+1,
      14+2,
      17+3,
      17+4,
      26+5,
      38+6,
      54+7,
      65+8,
      69+9
      ]

    dl = list()
    start = 0
    for r in rm:
        dl.extend(corrected_data_list[start:r])
        start = r + 1

    dl.extend(corrected_data_list[start:])
    dl_filterd = list()
    for data in dl:
        if data.label == 2:
            # print(data.label)
            # if data.label == 2:
            data.label = torch.tensor(1)
        dl_filterd.append(data)

    len(dl_filterd)
    return dl_filterd
            
    

# Raw Pearson

In [28]:
path = Path('graphs/raw_pearson/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)
train_dl[0]

C:\Users\mashh\AppData\Local\Temp\ipykernel_5624\129136280.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_5624\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


Data(x=[19, 153550], edge_index=[2, 361], edge_attr=[19, 19], label=0)

In [29]:
model = EEGGraphConvNetLSTM(input_size = 153550)

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [30]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 99303106


EEGGraphConvNetLSTM(
  (conv1): GCNConv(153550, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [31]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.893227	 acc=0.512821
Test  : epoch=1	 loss=0.703586	 acc=0.500000

Train : epoch=2	 loss=0.595126	 acc=0.602564
Test  : epoch=2	 loss=0.706039	 acc=0.500000

Train : epoch=3	 loss=0.599100	 acc=0.653846
Test  : epoch=3	 loss=0.714852	 acc=0.500000

Train : epoch=4	 loss=0.371745	 acc=0.666667
Test  : epoch=4	 loss=0.963897	 acc=0.500000

Train : epoch=5	 loss=0.300017	 acc=0.653846
Test  : epoch=5	 loss=1.313219	 acc=0.500000

Train : epoch=6	 loss=0.371194	 acc=0.666667
Test  : epoch=6	 loss=1.339589	 acc=0.500000

Train : epoch=7	 loss=0.341129	 acc=0.666667
Test  : epoch=7	 loss=1.377406	 acc=0.500000

Train : epoch=8	 loss=0.249509	 acc=0.717949
Test  : epoch=8	 loss=2.081392	 acc=0.500000

Train : epoch=9	 loss=0.233442	 acc=0.705128
Test  : epoch=9	 loss=3.586349	 acc=0.500000

Train : epoch=10	 loss=0.244910	 acc=0.692308
Test  : epoch=10	 loss=4.043225	 acc=0.500000



In [32]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## RAW PLI

In [33]:
path = Path('graphs/raw_pli/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_5624\129136280.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_5624\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [34]:
model = EEGGraphConvNetTemporal(input_size = 153550)

In [35]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 98842562


EEGGraphConvNetTemporal(
  (conv1): GCNConv(153550, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (conv4): GCNConv(256, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [16]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.958584	 acc=0.538462
Test  : epoch=1	 loss=12.992228	 acc=0.562500

Train : epoch=2	 loss=0.885180	 acc=0.589744
Test  : epoch=2	 loss=1.125366	 acc=0.500000

Train : epoch=3	 loss=1.017161	 acc=0.692308
Test  : epoch=3	 loss=0.802007	 acc=0.562500

Train : epoch=4	 loss=0.523012	 acc=0.692308
Test  : epoch=4	 loss=0.591864	 acc=0.437500

Train : epoch=5	 loss=0.388701	 acc=0.756410
Test  : epoch=5	 loss=1.572760	 acc=0.500000

Train : epoch=6	 loss=0.547139	 acc=0.692308
Test  : epoch=6	 loss=2.747724	 acc=0.500000

Train : epoch=7	 loss=0.338264	 acc=0.653846
Test  : epoch=7	 loss=3.141259	 acc=0.500000

Train : epoch=8	 loss=0.244018	 acc=0.653846
Test  : epoch=8	 loss=6.398689	 acc=0.500000

Train : epoch=9	 loss=0.293227	 acc=0.653846
Test  : epoch=9	 loss=5.214385	 acc=0.500000

Train : epoch=10	 loss=0.306737	 acc=0.653846
Test  : epoch=10	 loss=3.318988	 acc=0.500000



In [17]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,